# Load Modules and Database

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as aniplt
from tqdm import tqdm
import gc

In [2]:
database = "../database/prisoners_dilemma.json"
with open(database, "r") as database_file:
    data = pd.read_json(database_file)

# Pre Processing

In [3]:
def reduce_data(x: list[dict]) -> list[list]:
    y = [list(data.values()) for data in x]
    return y 

data["moves_data"] = data["moves_data"].apply(reduce_data)

# Visualising Player Growth

In [ ]:
def score_moves(player1_move: str, player2_move: str) -> tuple[int, int]:
    """
    This function maps the results into the scoreboard.

    Args:
        player1_move (str): The move made by the first player.
        player2_move (str): The move made by the second player.

    Returns:
        tuple[int,int]: A tuple of strings containing the points to be awarded to each player.
    """
    if (player1_move == "cooperate") and (player2_move == "cooperate"):
        return (2, 2)
    elif (player1_move == "cooperate") and (player2_move == "defect"):
        return (0, 3)
    elif (player1_move == "defect") and (player2_move == "cooperate"):
        return (3, 0)
    elif (player1_move == "defect") and (player2_move == "defect"):
        return (1, 1)
    else:
        print(
            f"Invalid move by the players: Player1: {player1_move}, Player2: {player2_move}. Please fix this."
        )
        exit(0)

def create_chart_element(ax, data):
    return ax.barh(data[0], data[1])

# Process data for round 1
# For each fixture in this round plot the progress of each player on a bar chart.
for round_number in list(data["round_id"].unique()):
    round_data = data[data["round_id"] == round_number]
    total_players = set(data.player2.unique()).union(set(data.player1.unique()))
    analysis_scoreboard = {}
    for player in total_players:
        analysis_scoreboard[player] = 0
    
    # Create a plot.
    figure, ax = plt.subplots()
    figure.set_figheight(10)
    figure.set_figwidth(15)
    ax.set_title(f"Round: {round}")
    ax.set_xlabel("Points")
    ax.set_ylabel("Strategies")
    
    # Create container for each stage and return back to the caller.
    strategy_wise_data = []
    for index in tqdm(range(len(round_data)), desc= f"Processing Game Data For Round {round_number}"):
        for move in round_data.iloc[index]["moves_data"]:
            strategy1_points, strategy2_points = score_moves(move[0], move[1])
            analysis_scoreboard[round_data.iloc[index]["player1"]] += strategy1_points
            analysis_scoreboard[round_data.iloc[index]["player2"]] += strategy2_points
            strategy_wise_data.append([
                list(analysis_scoreboard.keys()),
                list(analysis_scoreboard.values()),
            ])
    del round_data
    gc.collect()
    print("Creating plot data.")
    strategy_wise_data = [ax.barh(element[0], element[1]) for element in strategy_wise_data]
    print("Creating animation data.")
    animation_data = aniplt.ArtistAnimation(fig=figure, artists=strategy_wise_data, interval=1)
    del strategy_wise_data
    gc.collect()
    print("Saving animation to disk.")
    animation_data.save(
        f"../visuals/prisoners_dilemma_round{round_number}.mp4",
        writer=aniplt.FFMpegWriter(fps=30),
    )
    del animation_data
    gc.collect()

    # Reset the scoreboard.
    analysis_scoreboard = {}
    for player in total_players:
        analysis_scoreboard[player] = 0
    print(f"Round {round_number} analysis complete")

Processing Game Data For Round 0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:02<00:00, 17.78it/s]


Creating plot data.
Creating animation data.
Saving animation to disk.
